# SETUP

In [102]:
# IMPORTS
import logging
import requests
import json
from datetime import datetime
import os

# DEBUGGING
logging.basicConfig(level=logging.ERROR)

In [104]:
# GLOBALS
ARCHIVED = "false"
APIKEY = "API_KEY"
TEAMID = "TEAM_ID"

# ClickUp API Lookup Code

Most things are done in ClickUp (CU) via ID values, so we need a way to collect them into structures we can utilize during ticket creation.  This also helps avoid hardcoding ID values which can change w/ the addition of new teams, ticket statuses, etc.

In [234]:
class CULookup:
    """
    A class for handling ClickUp API interactions to manage spaces, folders, and lists.
    This class initializes with a predefined structure for ClickUp data and 
    provides methods to fetch and organize this data from the ClickUp API.
    """

    
    def __init__(self):
        """
        Initialize the CULookup instance with necessary data structures and API settings.
        
        Sets up:
        - A nested dictionary for organizing ClickUp data (lookup).
        - An empty dictionary for fields (which might be used later for field data).
        - Configuration for API requests, including whether to include archived items and API key.
        """
        # Initialize lookup dictionary with predefined structure
        self.lookup = {'evisit': {'spaces': {'P&E - General': {'id': 1304101, 'folders': {}}}}}
        
        # Initialize fields dictionary for storing field data
        self.fields = {}
        
        # Set whether to include archived items in API requests
        self.archived = ARCHIVED
        
        # Set API key for authentication
        self.apikey = APIKEY

        # Containers for the various CU lookup IDs and values
        self.lists = {}
        self.teams_cf = {}
        self.allocations_cf = {}
        self.tasktype_cf = {}
        self.release_cf = {}
        self.product_cf = {}
        self.figma_cf = {}
        self.fb_cf = {}
        self.trd_cf = {}
        self.specialization_cf = {}

    
    def perform_lookups(self):
        """
        Orchestrates the fetching of folder data for a specific space and updates the lookup structure.

        This method calls `lookup_folders()` to get the folder data for 'P&E - General' space 
        and then updates the class's `lookup` dictionary with this information.
        """
        # Fetch folders for the specified space and update the lookup dictionary
        folders = self.lookup_folders()
        self.lookup['evisit']['spaces']['P&E - General']['folders'] = folders        

    
    def lookup_folders(self):
        """
        Retrieve all folders for a specific space from ClickUp and return them structured.

        Args:
        None (uses predefined space_id)

        Returns:
        dict: A dictionary where keys are folder names and values are dictionaries 
              containing 'id' and 'lists' for each folder.

        This method constructs an API request to fetch all folders in a given space, 
        then processes the response to build a structured dictionary including 
        nested lists for each folder.
        """
        # Initialize dictionary to store folder data
        folders = {}

        # This is the P&E - General space
        space_id = "1304101"
        
        # Construct URL for fetching folders within a specific space
        url = "https://api.clickup.com/api/v2/space/" + space_id + "/folder"
        headers = {"Authorization": self.apikey}
        query = {"archived": self.archived}

        # Send GET request to fetch folder data
        response = requests.get(url, headers=headers, params=query)
        data = response.json()

        for f in data['folders']:
            # Store basic folder information in the dictionary
            folders[f['name']] = {'id': f['id']}
            
            # Lookup lists within each folder
            flist = self.lookup_lists(f['id'])
            folders[f['name']]['lists'] = flist
            
        return folders
        
    def lookup_lists(self, folder_id):
        """
        Fetch and return all lists within a specific ClickUp folder.
    
        Args:
        folder_id (str): The ID of the folder from which to retrieve lists.
    
        Returns:
        dict: A dictionary where keys are list names and values are dictionaries 
              containing the list's ID.
    
        This method constructs an API request to fetch all lists in the given folder, 
        including or excluding archived lists based on the 'archived' attribute. 
        It then processes the response to build a structured dictionary of lists.
        """
        # Initialize dictionary to store list data for a folder
        lists = {}
        
        # Construct URL for fetching lists within a specific folder
        url = "https://api.clickup.com/api/v2/folder/" + folder_id + "/list"
        headers = {"Authorization": self.apikey}
        query = {"archived": self.archived}

        # Send GET request to fetch list data
        response = requests.get(url, headers=headers, params=query)
        data = response.json()
        #print(data)
        #return

        for f in data['lists']:
            # Store basic list information in the dictionary
            lists[f['name']] = {'id': f['id']}

        return lists


    def lookup_fields(self, list_id):
        """
        Fetch all fields for a specific list from ClickUp and return them structured.
    
        Args:
        list_id (str): The ID of the list from which to retrieve fields.
    
        Returns:
        dict: A dictionary where keys are field names and values are dictionaries 
              containing 'id', 'type', and for dropdown fields, 'options'.
    
        This method constructs an API request to fetch all fields in a given list, 
        processes the response to build a structured dictionary, with special handling 
        for dropdown fields to include their options.
        """
        # Initialize an empty dictionary to store field data
        fields = {}
        
        # Construct the API endpoint URL for fetching list fields
        url = "https://api.clickup.com/api/v2/list/" + list_id + "/field"
        
        # Set up headers with the API key for authentication
        headers = {"Authorization": self.apikey, "Content-Type": "application/json"}
        
        # Define query parameters, including whether to include archived fields
        query = {"archived": self.archived}
    
        # Send GET request to the ClickUp API
        response = requests.get(url, headers=headers, params=query)
        
        # Parse the JSON response from the API
        data = response.json()
        #print(response)
        #return

        
        # Iterate through each field in the response data
        for f in data['fields']:
            # Store basic field information: name, ID, and type
            fields[f['name']] = {'id': f["id"], 'type': f["type"]}
            
            # Special handling for dropdown fields
            if f["type"] == "drop_down":
                opts = {}
                # Create a dictionary of options for dropdown fields
                for o in f["type_config"]["options"]:
                    opts[o["name"]] = {'id': o["id"]}
                
                # Add the options to the field's data
                fields[f['name']]["options"] = opts    
    
        # Update the instance variable with the newly fetched fields
        self.fields = fields
        
        # Return the dictionary of fields
        return fields
        
    
    def find_key(self, nested_dict, target_key):
        """
        Recursively search for a key in a nested dictionary structure.
    
        Args:
        nested_dict (dict): The dictionary to search through, which can be nested.
        target_key (str): The key you're looking for.
    
        Returns:
        list: A list of values associated with the target key. 
              If the key is not found, an empty list is returned.
    
        This method will search through nested dictionaries and lists to find all 
        instances where the target key exists, collecting their values.
        """
        # List to store all values associated with the target key
        results = []
    
        def search(d):
            # If the current item is a dictionary
            if isinstance(d, dict):
                for key, value in d.items():
                    # If the current key matches the target key, add its value to results
                    if key == target_key:
                        results.append(value)
                    # If the value is either a dict or a list, search within it
                    if isinstance(value, (dict, list)):
                        search(value)
            # If the current item is a list
            elif isinstance(d, list):
                # Recursively search each item in the list
                for item in d:
                    search(item)
    
        # Start the recursive search from the nested dictionary
        search(nested_dict)
        # Return the accumulated results
        return results

        
    def find_path_to_key(self, nested_dict, target_key):
        """
        Recursively find and return the path to a key in a nested dictionary.
    
        Args:
        nested_dict (dict): The nested dictionary to search through.
        target_key (str): The key to find the path for.
    
        Returns:
        list or None: A list representing the path to the key if found, 
                      or None if the key isn't in the dictionary.
    
        This method searches through nested structures to find the path to 
        the first occurrence of the target key.
        """
        def search(d, path):
            # Check if the current structure is a dictionary
            if isinstance(d, dict):
                for key, value in d.items():
                    # Extend the current path with the current key
                    new_path = path + [key]
                    # If the key matches the target, return the path
                    if key == target_key:
                        return new_path
                    # If the value is another dictionary or list, continue searching
                    if isinstance(value, (dict, list)):
                        result = search(value, new_path)
                        if result:
                            return result
            # Check if the current structure is a list
            elif isinstance(d, list):
                for i, item in enumerate(d):
                    # Extend the path with the list index and search the item
                    result = search(item, path + [i])
                    if result:
                        return result
            # If no match found, return None
            return None
        
        # Start the search from the root of the nested dictionary with an empty path
        path = search(nested_dict, [])
        return path

    
    def find(self, nested_dict, target_key):
        """
        Find both values and paths for a given key within a nested dictionary structure.
    
        Args:
        nested_dict (dict): The nested dictionary to search through.
        target_key (str): The key to search for in the dictionary.
    
        Returns:
        dict: A dictionary with two keys:
            - 'values': List of all values associated with the target key.
            - 'paths': List of paths to each occurrence of the target key within the nested structure.
    
        This method leverages two other methods:
        - `find_key`: To collect all values linked to the target key.
        - `find_path_to_key`: To gather all paths leading to the target key.
        """
        # Find all values associated with the target key
        vals = self.find_key(nested_dict, target_key)
        
        # Find all paths to the target key in the nested structure
        paths = self.find_path_to_key(nested_dict, target_key)
        
        # Combine values and paths into a single result dictionary
        result = {'values': vals, 'paths': paths}
    
        return result

    
    def create_lookup_library(self):
        """
        Populates dictionaries with IDs for ClickUp lists, sublists, and custom fields. This method:
        - Fetches and stores IDs for the Master Backlog list and its sublists.
        - Initializes custom field data for a specific strategic list.
        - Collects IDs for various custom fields like Team, Allocation, Task Type, Planned Release, Product, 
          Figma Link, Feature Brief, TRD Link, and Engineering Specialization, storing both the field ID 
          and, where applicable, the IDs of each option within those fields.
        This setup allows for quick reference and manipulation of ClickUp data structures in subsequent operations.
        """
        # Collect IDs for the Master Backlog list and its sublists
        tmp = self.lookup_lists('90140376790')
        for k, v in tmp.items():
            # Store the id of the main list
            self.lists[k] = v['id']
            if 'lists' in tmp[k]:
                # If there are sublists, store their IDs as well
                for kk, vv in tmp[k]['lists'].items():
                    self.lists[kk] = vv['id']
    
        # Initialize ClickUp Custom Fields (CF) data for Strategic Projects & VOC list
        self.lookup_fields('98198843')
    
        # Collect IDs for Team custom field
        self.teams_cf = self.create_cf("Team")
    
        # Collect IDs for Allocation custom field
        self.allocations_cf = self.create_cf("Allocation")
    
        # Collect IDs for Task Type custom field
        self.tasktype_cf = self.create_cf("Task Type")
    
        # Collect IDs for Planned Release custom field
        self.release_cf = self.create_cf("Planned Release")
    
        # Collect IDs for Product custom field
        self.product_cf = {}
        tmp = self.find(self.fields, "Product")
        # Note: Using index 2 here for the way this data structure is built
        self.product_cf['id'] = tmp['values'][2]['id']  
        for k, v in tmp['values'][2]['options'].items():
            # Store each option's ID for 'Product' field
            self.product_cf[k] = v['id']
    
        # Collect ID for Figma Link custom field
        tmp = self.find(self.fields, "Figma Link")
        self.figma_cf['id'] = tmp['values'][0]['id']  # Store the main field ID
    
        # Collect ID for Feature Brief custom field
        tmp = self.find(self.fields, "Feature Brief")
        self.fb_cf['id'] = tmp['values'][0]['id']  # Store the main field ID
    
        # Collect ID for TRD Link custom field
        tmp = self.find(self.fields, "TRD Link")
        self.trd_cf['id'] = tmp['values'][0]['id']  # Store the main field ID
    
        # Collect IDs for Engineering Specialization custom field
        self.specialization_cf = self.create_cf("Engineering Specialization")
        

    def create_cf(self, label):
        #Collect IDs for the CF
        cf = {}
        tmp = self.find(self.fields, label)
        cf['id'] = tmp['values'][0]['id']  # Store the main field ID
        for k, v in tmp['values'][0]['options'].items():
            # Store each option's ID for label field
            cf[k] = v['id']
        return cf

In [236]:
# Collect CU data
cu = CULookup()  # Create an instance of the CULookup class
cu.perform_lookups()  # Perform lookups to gather necessary data
cu.create_lookup_library()  # Create a library of lookup values for later use

### Examples

In [239]:
cu.find_path_to_key(cu.lookup, "Shield Team")

['evisit',
 'spaces',
 'P&E - General',
 'folders',
 'Master Backlog',
 'lists',
 'Shield Team']

In [241]:
cu.find(cu.lookup, "Shield Team")

{'values': [{'id': '901403893627'}],
 'paths': ['evisit',
  'spaces',
  'P&E - General',
  'folders',
  'Master Backlog',
  'lists',
  'Shield Team']}

In [243]:
cu.find_key(cu.lookup, "Shield Team")

[{'id': '901403893627'}]

In [245]:
print(cu.specialization_cf)

{'id': '57920976-d91a-45dc-b9b7-b0e84cedd9a2', 'Backend': 'efc13dde-8af8-4943-9054-a70edd923329', 'Frontend': '5478554b-e47d-4dec-82d1-8441bb0fb016', 'CDX': '33326811-f10e-4485-8443-5d9d2d41ab6f'}


In [247]:
print(cu.product_cf)

{'id': '58e1a6de-9649-4bdd-aba2-df8735484944', 'eVisit': '890882a2-1f1b-4659-83d0-0224226e39ac', 'Bluestream Health': '40314f07-e952-4791-bd06-db114671cb53', 'eVisit Consult': '3005b97e-78ae-4fab-ba15-4ee80859481d'}


# ClickUp API Ticket Creation Class

In [121]:
class TaskMaker2000:
    """
    A class for creating tickets in ClickUp using their API.

    This class facilitates the creation of new tasks in ClickUp by providing 
    an interface to interact with ClickUp's task creation endpoint.
    """

    def __init__(self):
        """
        Initialize the TaskMaker2000 instance.

        Currently, this method does nothing but can be extended for initialization logic.
        """
        pass

    
    def create_ticket(self, task):
        """
        Create a new task in ClickUp for the given list.

        Args:
        task (dict): A dictionary containing all necessary task details. 
                     Expected keys include:
                     - 'list_id': ID of the list where the task should be added
                     - 'name': Task name
                     - 'markdown_description': Description in markdown format
                     - 'assignees': List of user IDs to assign the task to
                     - 'group_assignees': List of group IDs to assign the task to
                     - 'tags': List of tags for the task
                     - 'status': Status of the task
                     - 'priority': Priority level
                     - 'due_date': Due date for the task (string format)
                     - 'points': Points for the task, if applicable
                     - 'parent': Parent task ID if this is a subtask
                     - 'links_to': List of tasks or items this task links to
                     - 'custom_fields': Dictionary of custom field IDs and values
                     - more to follow...

        Returns:
        None. Prints the response from the API, either as JSON or raw if JSON fails.

        Raises:
        Any exception from the requests module will be caught and the raw response printed.
        """
        # Set up authentication headers for the API request
        headers = {'Authorization': APIKEY}
        
        # Construct the URL for posting a new task to ClickUp
        url = "https://api.clickup.com/api/v2/list/" + task['list_id'] + "/task"
        query = {"custom_task_ids": "true", "team_id": TEAMID}
        
        # Prepare the JSON data for the new task
        json_data = {
            "name": task['name'],
            "markdown_description": task['markdown_description'],
            "assignees": task['assignees'],
            "archived": False,
            "group_assignees": task['group_assignees'],
            "tags": task['tags'],
            "status": task['status'],
            "priority": task['priority'],
            "due_date": task['due_date'],
            "due_date_time": False,
            "time_estimate": None,
            "start_date": None,
            "start_date_time": False,
            "points": task['points'],
            "notify_all": False,
            "parent": task['parent'],
            "links_to": task['links_to'],
            "check_required_custom_fields": False,
            "custom_fields": task['custom_fields']
        }

        # Send POST request to create the task
        response = requests.post(url, headers=headers, json=json_data)
        
        # Try to format and print the response as JSON
        try:
            #json_object = json.dumps(response.json(), indent=4) 
            return response.json()
        except:
            # If JSON formatting fails, print the raw response
            return response

# CREATING THE TICKETS

In [123]:
# PARSE TICKET JSON, INITIALIZE THE CULOOKUP OBJECT, CREATE TICKETS, PRINT RESULT REPORT

# Imports
import json  # Import the JSON module to work with JSON data

# Local variables
err = False  # Initialize an error flag to track if any errors occur
target_cu_list = 'Strategic Proj. & VOC'  # Define the target list for ClickUp (CU) tickets
parent_cu_ticket = '86b1pwczb'  # Define the parent ticket ID to add new tickets to
created_tickets = {}  # Dictionary to store successfully created tickets
skipped_tickets = {}  # Dictionary to store tickets that could not be created

# Opening JSON file with ticket data
data_file = 'data/Systems_Data_Mapping_sprint_tickets_for_clickup.json'  # Specify the path to the JSON file containing ticket data
try:
    f = open(data_file)  # Attempt to open the JSON file

    # Returns JSON object as a dictionary
    data = json.load(f)  # Load the JSON data into a Python dictionary
    
    # Closing file
    f.close()  # Close the file after loading the data
except:
    # Handle any errors that occur while opening the file
    print(f"Error opening {data_file}.  Aborting.")  # Print an error message
    err = True  # Set the error flag to True

if err == False:  # Proceed only if no errors occurred importing the JSON data
    # Info
    print(f"Processing `{data_file}` with {len(data)} records.")  # Print the number of records being processed

    for i in range(len(data)):  # Loop through each record in the data
        # Assign next record from the JSON ticket data
        t = data[i]  # Get the current ticket record
    
        # Status message for next record in the loop
        print(f"Processing record {i} with title '{t['Ticket Title']}'")  # Print the title of the current ticket
        
        # Apply manual fixes for bad data...
        t['Release'] = "March 2025"  # Set a default release date for the ticket
    
        # Populate the ticket details to be sent to the CU API
        task = {
            "list_id": cu.lists[target_cu_list],  # Get the list ID for the target CU list
            "name": t['Ticket Title'],  # Set the ticket title
            "markdown_description": t['Ticket Description'],  # Set the ticket description
            "assignees": None,  # No specific assignees
            "group_assignees": None,  # No group assignees
            "tags": [],  # Initialize an empty list for tags
            "status": "READY FOR DEVELOPMENT",  # Set the initial status of the ticket
            "priority": 3,  # Set the priority level
            "due_date": None,  # No due date specified
            "points": t['Points'],  # Set the points for the ticket
            "parent": parent_cu_ticket,  # Set the parent ticket ID
            "links_to": None,  # No links specified
            "custom_fields": [  # Populate custom fields with relevant data
                {"id": cu.allocations_cf['id'], "value": cu.allocations_cf[t['Allocation']]},
                {"id": cu.teams_cf['id'], "value": cu.teams_cf[t['Team']]},
                {"id": cu.tasktype_cf['id'], "value": cu.tasktype_cf[t['Task Type']]},
                {"id": cu.release_cf['id'], "value": cu.release_cf[t['Release']]},
                {"id": cu.product_cf['id'], "value": cu.product_cf[t['Product']]},
                {"id": cu.figma_cf['id'], "value": t['Figma Link']},
                {"id": cu.specialization_cf['id'], "value": cu.specialization_cf[t['Specialization']]},
                {"id": cu.fb_cf['id'], "value": t['Feature Brief']},
            ]
        }
    
        # Create ticket maker class
        tm = TaskMaker2000()  # Instantiate the TaskMaker2000 class to handle ticket creation
        
        # Create the new ticket and assign the response to a variable for processing
        result = tm.create_ticket(task)  # Call the create_ticket method with the task details
    
        # Logic to handle ticket creation results
        if result.get('err') is not None:  # Check if there was an error in ticket creation
            skipped_tickets[t['Ticket Title']] = result  # Store the error result in skipped_tickets
        else:
            created_tickets[result['name']] = result['id']  # Store the successfully created ticket

    # Print post-import report
    print("\n**All imports complete!**")  # Indicate that the import process is complete
    print("Import errors list:")  # Print header for errors
    for k, v in skipped_tickets.items():  # Loop through skipped tickets
        print(f"\t * {k} :: {v}")  # Print each skipped ticket and its error
    print("Import successful list:")  # Print header for successful imports
    for k, v in created_tickets.items():  # Loop through created tickets
        print(f"\t * {k} :: {v}")  # Print each successfully created ticket and its ID

Processing `data/Systems_Data_Mapping_sprint_tickets_for_clickup.json` with 18 records.
Processing record 0 with title 'BE - Create new table for field categories'
Processing record 1 with title 'BE - Build logic to search field definitions'
Processing record 2 with title 'BE - Update Dart Reporting for eData'
Processing record 3 with title 'BE - Build logic to store and retrieve mapped data'
Processing record 4 with title 'BE - Create migration to load initial system data fields'
Processing record 5 with title 'BE - Add endpoint to retrieve available field Categories'
Processing record 6 with title 'BE - System fields in forms'
Processing record 7 with title 'FE - Update Fields Library 2 columns'
Processing record 8 with title 'FE - Modify Fields Library form'
Processing record 9 with title 'FE - Disallow field type modification for system fields'
Processing record 10 with title 'FE - Add tooltip to show if the field contains PHI'
Processing record 11 with title 'FE - Add sort functio

In [125]:
# WRITE TICKET CREATION RESULTS REPORT TO DISK

# Get the current date and time
now = datetime.now()
counter = 1  # Initialize a counter to keep track of file versions

# Create the initial filename using the current date and the counter
fname = now.strftime("%Y-%m-%d") + "_ticket_creation_results_" + str(counter) + ".txt"

# Check if the file with the current filename already exists
while os.path.exists(fname):
    counter += 1  # Increment the counter if the file exists
    # Create a new filename with the updated counter
    fname = now.strftime("%Y-%m-%d") + "_ticket_creation_results_" + str(counter) + ".txt"
    
# Open the file for writing.
with open(fname, "w") as f:
    # Write a header indicating that all imports are complete
    f.write("**All imports complete!**\n\n")
    
    # Write a section header for import errors
    f.write("Import errors list:\n")
    # Iterate over the skipped_tickets dictionary and write each key-value pair to the file
    for k, v in skipped_tickets.items():
        f.write(f"\t * {k} :: {v}\n")  # Format the output for each skipped ticket
    
    # Write a section header for successful imports
    f.write("Import successful list:\n")
    # Iterate over the created_tickets dictionary and write each key-value pair to the file
    for k, v in created_tickets.items():
        f.write(f"\t * {k} :: {v}\n")  # Format the output for each successfully created ticket

# Adding Dependencies

In [173]:
# Track if we have a JSON error
err = False

# Path to the JSON file containing ticket data
data_file = 'data/Forms_sprint_tickets_for_clickup_new.json'  

try:
    f = open(data_file)  # Attempt to open the JSON file

    # Returns JSON object as a dictionary
    data = json.load(f)  # Load the JSON data into a Python dictionary
    
    # Closing file
    f.close()  # Close the file after loading the data
except:
    # Handle any errors that occur while opening the file
    print(f"Error opening {data_file}. Aborting.")  # Print an error message
    err = True  # Exit the script if file cannot be opened


# Add dependency info to the CU tickets if the JSON import suceeded
if err == False:
    for t in range(len(data)):
        # Pull ticket title
        ticket = data[t]['Ticket Title']
        # Skip over tickets with no dependency information
        try:
            # Split the dependency string into items, stripping any extra whitespace
            items = [x.strip() for x in data[t]['Ticket Depends On'].split(",")]
            
            print(f"- Adding {len(items)} dependencies for {ticket}.")
            
            # Create API URL; assumes 'data_depends' is a dictionary mapping ticket titles to task IDs
            url = f"https://api.clickup.com/api/v2/task/{data_depends[ticket]}/dependency"
            
            # For each dependency, create payload and submit
            for i in items:
                # Construct payload for API call
                payload = { "depends_on": data_depends[i] }
                headers = {
                    "accept": "application/json",
                    "content-type": "application/json",
                    "Authorization": "API_KEY"
                }
                # Make the call to the CU API and record the response
                #response = requests.post(url, json=payload, headers=headers)
                print(f"\t- Created dependency to {i}")
        except:
            # If there's no 'Ticket Depends On' key or other errors, this will catch them
            print(f"- No dependencies for {ticket}.")
        finally:
            # Print a new line for better readability in console output
            print("\n")

- Adding 5 dependencies for FE - Publish/Unpublish.
	- Created dependency to FE - Component Pubish/Unpublish Modal Dialog
	- Created dependency to FE - Component Publish/Unpublish toggle
	- Created dependency to BE - Add new fields to CustomForm
	- Created dependency to BE - Update CustomFormController to handle new fields
	- Created dependency to BE - Add set_published endpoint to CustomFormController


- No dependencies for FE - Component Pubish/Unpublish Modal Dialog.


- No dependencies for FE - Component Publish/Unpublish toggle.


- No dependencies for BE - Add new fields to CustomForm.


- No dependencies for BE - Update CustomFormController to handle new fields.


- No dependencies for BE - Add set_published endpoint to CustomFormController.


- Adding 4 dependencies for FE - Form Preferences.
	- Created dependency to FE - Component - Modify ComplexListItems to accept radio buttons
	- Created dependency to FE - Component - Checkboxes Tree
	- Created dependency to FE - Component

In [162]:
# WRITE DEPENDENCY LINKAGE RESULTS REPORT TO DISK

# Get the current date and time
now = datetime.now()
counter = 1  # Initialize a counter to keep track of file versions

# Create the initial filename using the current date and the counter
fname = now.strftime("%Y-%m-%d") + "_dependency_linking_results_" + str(counter) + ".txt"

# Check if the file with the current filename already exists
while os.path.exists(fname):
    counter += 1  # Increment the counter if the file exists
    # Create a new filename with the updated counter
    fname = now.strftime("%Y-%m-%d") + "_dependency_linking_results_" + str(counter) + ".txt"
    
# Open the file for writing.
with open(fname, "w") as f:
    # Write a header indicating that all imports are complete
    f.write("**All dependency linking complete!**\n\n")
    
    # Write a section header for import errors
    f.write("Dependency linking errors list:\n")
    # Iterate over the skipped_tickets dictionary and write each key-value pair to the file
    for k, v in skipped_tickets.items():
        f.write(f"\t * {k} :: {v}\n")  # Format the output for each skipped ticket
    
    # Write a section header for successful dependency linkage
    f.write("\nDependency linking successful list:\n")
    # Iterate over the dependency linkage dictionary and write each key-value pair to the file
    for k, v in created_tickets.items():
        f.write(f"\t * {k} :: {v}\n")  # Format the output for each successfully created ticket